# ESTUDIO DE DATOS ABIERTOS 

Bolsas excrementos perros

http://datos.gob.es/es/catalogo/l01280796-mobiliario-urbano-papeleras-con-dispensador-de-bolsas-de-excrementos-para-perros

Radares Madrid (necesario preprocesamiento)

http://datos.gob.es/es/catalogo/l01280796-radares1

https://www.motorpasion.com/industria/todos-los-radares-de-madrid

http://aeaclub.org/radares-velocidad-madrid/

http://aeaclub.org/radares-madrid/

https://www.multascea.es/recurrir/64-sabes-que-radares-se-han-activado-en-madrid

Incidencias M30

http://datos.gob.es/es/catalogo/l01280796-trafico-calle-30-incidencias1


Accidentes barcelona

http://datos.gob.es/es/catalogo/l01080193-personas-involucradas-en-accidentes-gestionados-por-la-guardia-urbana-en-la-ciudad-de-barcelona

http://datos.gob.es/es/catalogo/l01080193-accidentes-gestionados-por-la-guardia-urbana-en-la-ciudad-de-barcelona

# Importo librerías 

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn import datasets, linear_model
import sklearn
from sklearn.ensemble import IsolationForest
import geopy
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import logging
from sklearn.model_selection import train_test_split
import time
from datetime import timedelta
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
from plotly import tools
py.offline.init_notebook_mode(connected=True)
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from h3 import h3
import folium
from folium.plugins import MarkerCluster
from IPython import display

# Mapa de papeleras de Madrid con expendedor de bolsas: 

In [2]:
papeleras_madrid = pd.read_csv(
    'papeleras_con_expendedor_de_bolsas.csv', encoding='latin-1', sep=';')

In [3]:
papeleras_madrid[:2]

,COD_TIPO_DATO,TIPO_DATO,IDENTIFICADOR_MINT,COD_AREA,COD_NDP,DIRECCION_COMPLETA,DIRECCION AUXILIAR,COD_VIAL,CLASE_VIAL,PARTICULAR_VIAL,NOMBRE_VIAL,TIPO_NÚMERO,NÚMERO,COD_BARRIO,BARRIO,COD_DISTRITO,DISTRITO,UTMX,UTMY,LONGITUD,LATITUD,FECHA_INSTALACION_O_SUSTITUCION,MODELO,Unnamed: 23
0,5001,PAPELERA_EXP,504757,NaN,20004152,AVENIDA DE LA GRAN VIA DE SAN FRANCISCO N.19,"AVENIDA DE LA GRAN VIA DE SAN FRANCISCO,19",669050,AVENIDA,DE LA,GRAN VIA DE SAN FRANCISCO,N,19,11,PALACIO,1,CENTRO,"439424,8287","4473496,159",-3.71393,40.40988,16/07/2010,50_LTS_EX,NaN
1,5001,PAPELERA_EXP,516240,NaN,20147367,AVENIDA DE LA GRAN VIA DE SAN FRANCISCO N.6,"AVENIDA DE LA GRAN VIA DE SAN FRANCISCO,6",669050,AVENIDA,DE LA,GRAN VIA DE SAN FRANCISCO,N,6,11,PALACIO,1,CENTRO,"439530,2964","4473316,587",-3.71267,40.40827,21/02/2011,50_LTS_EX,NaN


In [4]:
lat = papeleras_madrid['LATITUD']
lng = papeleras_madrid['LONGITUD']
m = folium.Map(
    location=[sum(lat) / len(lat), sum(lng) / len(lng)],
    zoom_start=13,
    tiles='stamentoner')

In [5]:
papeleras_madrid.shape

(6042, 24)

In [6]:
marker_cluster = MarkerCluster().add_to(m)
for index, row in papeleras_madrid.iterrows():
    mk = folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        icon=folium.Icon(color='lightgreen', icon='remove-sign'),
    ).add_to(marker_cluster)
#    m.add_child(mk)

In [7]:
m.save(outfile='mapa_papeleras.html')

# Datos de accidentes en Barcelona:

In [62]:
accidentes_barna = pd.read_csv('2017_accidents_gu_bcn.csv', sep=',')

In [63]:
accidentes_barna[:1]

,Número d'expedient,Codi districte,Nom districte,Codi barri,Nom barri,Codi carrer,Nom carrer,Num postal caption,Descripció dia setmana,Dia setmana,Descripció tipus dia,NK Any,Mes de any,Nom mes,Dia de mes,Hora de dia,Descripció torn,Descripció causa vianant,Número de morts,Número de lesionats lleus,Número de lesionats greus,Número de víctimes,Número de vehicles implicats,Coordenada UTM (X),Coordenada UTM (Y),Longitud,Latitud
0,2017S008429,-1,Desconegut,-1,Desconegut,-1,Número 27 ...,Desconegut,Divendres,Dv,Laboral,2017,10,Octubre,13,8,Matí,Altres,0,2,0,2,2,426936.65,4577079.15,2.125624,41.340045


In [64]:
lat = accidentes_barna['Latitud']
lng = accidentes_barna['Longitud']
m = folium.Map(
    location=[sum(lat) / len(lat), sum(lng) / len(lng)],
    zoom_start=13,
    tiles='stamentoner')

In [65]:
accidentes_barna.shape

(10339, 27)

In [66]:
marker_cluster = MarkerCluster().add_to(m)
for index, row in accidentes_barna.iterrows():
    if row["Número de morts"]>0:
        mk = folium.Marker(
            location=[row['Latitud'], row['Longitud']],
                popup=u'Número de expediente: '+str(row["Número d'expedient"])
                        +'<br>Lesionados leves: '+str(row["Número de lesionats lleus"])
                        +'<br>Lesionados graves: '+str(row["Número de lesionats greus"])
                        +'<br>Muertes: '+str(row["Número de morts"])
                        +'<br>Vehículos implicados: '+str(row["Número de vehicles implicats"])
                        +'<br>Día de la semana: '+str(row["Descripció dia setmana"])
                        +'<br>Tipo de día: '+str(row["Descripció tipus dia"]),
            icon=folium.Icon(color='red', icon='remove-sign'),
        )#.add_to(marker_cluster)
        m.add_child(mk)
    elif row["Número de lesionats greus"]>0:
        mk = folium.Marker(
            location=[row['Latitud'], row['Longitud']],
                popup=u'Número de expediente: '+str(row["Número d'expedient"])
                        +'<br>Lesionados leves: '+str(row["Número de lesionats lleus"])
                        +'<br>Lesionados graves: '+str(row["Número de lesionats greus"])
                        +'<br>Muertes: '+str(row["Número de morts"])
                        +'<br>Vehículos implicados: '+str(row["Número de vehicles implicats"])
                        +'<br>Día de la semana: '+str(row["Descripció dia setmana"])
                        +'<br>Tipo de día: '+str(row["Descripció tipus dia"]),
            icon=folium.Icon(color='orange', icon='remove-sign'),
        ).add_to(marker_cluster)
        #m.add_child(mk)
    else:
        mk = folium.Marker(
            location=[row['Latitud'], row['Longitud']],
                popup=u'Número de expediente: '+str(row["Número d'expedient"])
                        +'<br>Lesionados leves: '+str(row["Número de lesionats lleus"])
                        +'<br>Lesionados graves: '+str(row["Número de lesionats greus"])
                        +'<br>Muertes: '+str(row["Número de morts"])
                        +'<br>Vehículos implicados: '+str(row["Número de vehicles implicats"])
                        +'<br>Día de la semana: '+str(row["Descripció dia setmana"])
                        +'<br>Tipo de día: '+str(row["Descripció tipus dia"]),
            icon=folium.Icon(color='lightgreen', icon='remove-sign'),
        ).add_to(marker_cluster)

In [67]:
m.save(outfile='accidentes_barna.html')